# Tutorial

This is a tutorial for MB-MVPA using task-fMRI data of Mixed-gamble task by Tom et al., 2007. 



### Import the MB-MVPA libarary.

Other libraries(nilean, keras, etc..) dosen't need to be imported.<br>
Because mb-mvpa has wrapping the libararies.<br>
You don't necessarily have to know fMRI libraries like nilearn and machine learning libraries like tensorflow.<br>
<b>MB-MVPA is all you need.</b>

Most of mb-mvpa are wrapping nilearn, tensorflow, Keras and etc., so warning can occur from that libraries.<br>
This page does not print warning because most of them are can be ignored.<br>
You don't need to remove the warning when you are actually using it.

In [1]:
from time import perf_counter

In [2]:
from mbmvpa.preprocessing.preprocess import DataPreprocessor

/home/cheoljun/.cache/pypoetry/virtualenvs/model-based-fmri-A7ELC43w-py3.7/lib/python3.7/site-packages/nilearn/glm/__init__.py:56: FutureWarning: The nilearn.glm module is experimental. It may change in any future release of Nilearn.
  'It may change in any future release of Nilearn.', FutureWarning)


TODO: add original data download link

Data download from AWS S3, ~ <b>1GB</b> (would be under the "Mixed-gamble_task/example_data/").<br>

We provide a small subset (2 subjects) of original Tom's dataset (16 subjects). The fMRI images in the example is preprocessed by conventional fMRI preprocessing pipeline by using 
[*fmriprep*](https://fmriprep.org/en/stable/) v.20.1.0. Please refer to the [original](https://openneuro.org/datasets/ds000005/versions/00001) for more information.

In [3]:
#root = load_example_data("tom")
root = "/data2/project_modelbasedMVPA/ds000005"

### Preprocessing fMRI images and behavioral data

MB-MVPA requires primariliy preprocessed task-fMRI experiments data fromatted in conventional [BIDS format](https://bids-specification.readthedocs.io/en/stable/) 

It expects the following organized files. All the naming conventions used here conform with outputs from *fmriprep* v.20.1.0. by Poldrack lab.

The fMRI images are usually located here<br>
<i>{BIDS_ROOT}/derivatives/fmriprep/subject/session/run/func/*nii.gz</i><br>
And the behavior data are located here<br>
<i>{BIDS_ROOT}/subject/session/run/func/*.tsv</i>

In [4]:
s = perf_counter()

dm_model = 'ra_prospect'

def example_adjust(row):
    ## rename data in a row to the name which can match hbayesdm.ra_prospect requirements ##
    row["gamble"] = 1 if row["respcat"] == 1 else 0
    row["cert"] = 0
    return row

def example_filter(row):
    # include all trial data
    return True

def example_latent(row, param_dict):
    ## calculate subjectives utility for choosing Gamble over Safe option
    ## prospect theory with loss aversion and risk aversion is adopted
    modulation = (row["gain"] ** param_dict["rho"]) - (param_dict["lambda"] * (row["loss"] ** param_dict["rho"]))
    row["modulation"] = modulation
    return row


preprocessor = DataPreprocessor(bids_layout=root,
                               adjust_function=example_adjust,
                               filter_function=example_filter,
                               latent_function=example_latent,
                               dm_model=dm_model,
                               confounds=['a_comp_cor_00',
                                         'a_comp_cor_01',
                                         'a_comp_cor_02',
                                         'a_comp_cor_03',
                                         'a_comp_cor_04',
                                         'a_comp_cor_05',
                                          "trans_x", 
                                          "trans_y",
                                          "trans_z",
                                          "rot_x",
                                          "rot_y", 
                                          "rot_z"], 
                               zoom=(2,2,2))
print(f"elapsed time: {(perf_counter()-s) / 60:.2f} minutes")

elapsed time: 0.49 minutes


In [5]:
preprocessor.summary()

[  fMRIPrep  ] BIDS Layout: .../ds000005/derivatives/fmriprep | Subjects: 16 | Sessions: 0 | Runs: 48
[  MB-MVPA   ] BIDS Layout: ...PA/ds000005/derivatives/mbmvpa | Subjects: 16 | Sessions: 0 | Runs: 48


In [6]:
s = perf_counter()

#preprocessor.preprocess(overwrite=True,core=16)
#preprocessor.preprocess(overwrite=True,core=16)
#feature_name = 'voxelfeaturemasked'
preprocessor.X_generator.run(overwrite=True)
print(f"elapsed time: {(perf_counter()-s) / 60:.2f} minutes")

12it [03:17, 16.43s/it]

elapsed time: 3.42 minutes


In [7]:
preprocessor.summary()

[  fMRIPrep  ] BIDS Layout: .../ds000005/derivatives/fmriprep | Subjects: 16 | Sessions: 0 | Runs: 48
[  MB-MVPA   ] BIDS Layout: ...PA/ds000005/derivatives/mbmvpa | Subjects: 16 | Sessions: 0 | Runs: 48


### Load data and shape check

In [2]:
from mbmvpa.data.loader import BIDSDataLoader

In [3]:
root = "/data2/project_modelbasedMVPA/ds000005"

In [4]:
s = perf_counter()

#loader = BIDSDataLoader(layout=root,feature_name=feature_name)
loader = BIDSDataLoader(layout=root)
X,y = loader.get_total_data()

print(f"elapsed time: {(perf_counter()-s) / 60:.2f} minutes")

/home/cheoljun/.cache/pypoetry/virtualenvs/model-based-fmri-A7ELC43w-py3.7/lib/python3.7/site-packages/bids/layout/models.py:152: FutureWarning: The 'extension' entity currently excludes the leading dot ('.'). As of version 0.14.0, it will include the leading dot. To suppress this warning and include the leading dot, use `bids.config.set_option('extension_initial_dot', True)`.
  FutureWarning)


elapsed time: 0.17 minutes


In [5]:
print("X", X.shape)
print("y", y.shape)

X (11520, 571)
y (11520,)


In [6]:
voxel_mask = loader.get_voxel_mask()

### Fitting MVPA models & Results

In [8]:
from mbmvpa.models.mvpa_mlp import MLP

In [9]:
MVPA_model = MLP(X=X,
                y=y,
                layer_dims=[512,256,128],
                activation='relu',
                n_patience=10,
                n_repeat=10,
                voxel_mask = voxel_mask)



In [10]:
s = perf_counter()

coeffs = MVPA_model.run()

print(f"elapsed time: {(perf_counter()-s) / 60:.2f} minutes")

[001] - score: 0.1686 p: 0.0000
[002] - score: 0.1092 p: 0.0000
[003] - score: 0.1245 p: 0.0000
[004] - score: 0.1340 p: 0.0000
[005] - score: 0.1301 p: 0.0000
[006] - score: 0.1410 p: 0.0000
[007] - score: 0.1449 p: 0.0000
[008] - score: 0.1274 p: 0.0000
[009] - score: 0.1109 p: 0.0000
[010] - score: 0.1089 p: 0.0000
[011] - score: 0.1321 p: 0.0000
[012] - score: 0.1530 p: 0.0000
[013] - score: 0.1708 p: 0.0000
[014] - score: 0.1473 p: 0.0000
[015] - score: 0.1266 p: 0.0000
elapsed time: 10.30 minutes


In [11]:
MVPA_model = MLP(X=X,
                y=y,
                layer_dims=[256,256],
                activation='relu',
                n_patience=10,
                n_repeat=10,
                voxel_mask = voxel_mask)

coeffs = MVPA_model.run()


[001] - score: 0.1591 p: 0.0000
[002] - score: 0.1049 p: 0.0000
[003] - score: 0.1135 p: 0.0000
[004] - score: 0.1374 p: 0.0000
[005] - score: 0.1413 p: 0.0000
[006] - score: 0.0882 p: 0.0001
[007] - score: 0.1501 p: 0.0000
[008] - score: 0.1279 p: 0.0000
[009] - score: 0.1015 p: 0.0000
[010] - score: 0.0866 p: 0.0001


In [12]:
MVPA_model = MLP(X=X,
                y=y,
                layer_dims=[256],
                activation='relu',
                n_patience=10,
                n_repeat=10,
                voxel_mask = voxel_mask)

coeffs = MVPA_model.run()


[001] - score: 0.0976 p: 0.0000
[002] - score: 0.1032 p: 0.0000
[003] - score: 0.1078 p: 0.0000
[004] - score: 0.1266 p: 0.0000
[005] - score: 0.1154 p: 0.0000
[006] - score: 0.1082 p: 0.0000
[007] - score: 0.1127 p: 0.0000
[008] - score: 0.1131 p: 0.0000
[009] - score: 0.1066 p: 0.0000
[010] - score: 0.0859 p: 0.0001


In [14]:
MVPA_model = MLP(X=X,
                y=y,
                layer_dims=[256,128,64,32],
                dropout_rate=0.1,
                activation='relu',
                n_patience=10,
                n_repeat=10,
                voxel_mask = voxel_mask)

coeffs = MVPA_model.run()


[001] - score: 0.1322 p: 0.0000
[002] - score: 0.0817 p: 0.0003
[003] - score: 0.1381 p: 0.0000
[004] - score: 0.1036 p: 0.0000
[005] - score: 0.0662 p: 0.0031
[006] - score: 0.0779 p: 0.0005
[007] - score: 0.0959 p: 0.0000
[008] - score: 0.0784 p: 0.0005
[009] - score: 0.0383 p: 0.0869
[010] - score: 0.0866 p: 0.0001


In [16]:
MVPA_model = MLP(X=X,
                y=y,
                layer_dims=[1024,1024],
                dropout_rate=0.5,
                activation='relu',
                n_patience=10,
                n_repeat=10,
                voxel_mask = voxel_mask)

coeffs = MVPA_model.run()


[001] - score: 0.1684 p: 0.0000
[002] - score: 0.1190 p: 0.0000
[003] - score: 0.1221 p: 0.0000
[004] - score: 0.1703 p: 0.0000
[005] - score: 0.1353 p: 0.0000
[006] - score: 0.1165 p: 0.0000
[007] - score: 0.1324 p: 0.0000
[008] - score: 0.1173 p: 0.0000
[009] - score: 0.1196 p: 0.0000
[010] - score: 0.0912 p: 0.0000


In [17]:
MVPA_model = MLP(X=X,
                y=y,
                layer_dims=[2048,2048],
                dropout_rate=0.5,
                activation='relu',
                n_patience=10,
                n_repeat=10,
                voxel_mask = voxel_mask)

coeffs = MVPA_model.run()


[001] - score: 0.1551 p: 0.0000
[002] - score: 0.1120 p: 0.0000
[003] - score: 0.1435 p: 0.0000
[004] - score: 0.1673 p: 0.0000
[005] - score: 0.1567 p: 0.0000
[006] - score: 0.1216 p: 0.0000
[007] - score: 0.1105 p: 0.0000
[008] - score: 0.0644 p: 0.0039
[009] - score: 0.1067 p: 0.0000
[010] - score: 0.0951 p: 0.0000


In [20]:
MVPA_model = MLP(X=X,
                y=y,
                layer_dims=[1],
                dropout_rate=0.0,
                activation='linear',
                n_patience=10,
                n_repeat=10,
                voxel_mask = voxel_mask)

coeffs = MVPA_model.run()


[001] - score: 0.1086 p: 0.0000
[002] - score: 0.0928 p: 0.0000
[003] - score: 0.0791 p: 0.0004
[004] - score: 0.1257 p: 0.0000
[005] - score: 0.1094 p: 0.0000
[006] - score: 0.1143 p: 0.0000
[007] - score: 0.0571 p: 0.0106
[008] - score: 0.0856 p: 0.0001
[009] - score: 0.0752 p: 0.0008
[010] - score: 0.0801 p: 0.0003


In [15]:
s = perf_counter()

sham_errors = MVPA_model.sham()

print(f"elapsed time: {(perf_counter()-s) / 60:.2f} minutes")

[001] - score: -0.0213 p: 0.3407
[002] - score: 0.0251 p: 0.2617
[003] - score: -0.0141 p: 0.5285
[004] - score: 0.0107 p: 0.6315


KeyboardInterrupt: 

In [ ]:
img = MVPA_model.image(save_path='.', task_name='example')